In [31]:
import pandas as pd
import numpy as np
import ipywidgets as w
from ipywidgets import HBox, VBox
from ipywidgets import Layout, widgets
from IPython.display import display, IFrame, HTML
from __future__ import print_function
from utils import Util
from rbm import RBM
import math, re, datetime as dt, glob
from urllib.parse import quote
from urllib.request import Request, urlopen
from google_images_download import google_images_download
from nltk.corpus import wordnet
from PIL import Image

In [3]:
att_df = pd.read_json('etl/attractions.json',orient='records')

In [4]:
name = w.Text(description="User Name")
place = w.Text(description="Destination")
budget = w.IntRangeSlider(min=att_df.price.min(), max=att_df.price.max(),step=10,value=[att_df.price.min(),att_df.price.max()],description="Budget")
out = w.VBox([name,place,budget])
display(out)

In [5]:
start = w.DatePicker(description='Start Date',disabled=False)
display(start)
end = w.DatePicker(description='End Date',disabled=False)
display(end)

DatePicker(value=None, description='Start Date')

DatePicker(value=None, description='End Date')

In [6]:
if(end.value < start.value):
    print("Wrong end date! Please select an end date later than the start date.")

In [5]:
category_df = att_df.groupby('category').size().reset_index().sort_values([0],ascending=False)[:18]
categories = list(category_df.category.values)
cat_rat = dict()
def on_button_clicked(b):
    if b.description in cat_rat:
        return
    print(b.description)
    slider = w.IntSlider(min=0,max=5,step=1,description='Rate')
    display(slider)
    cat_rat[b.description] = slider
    if(len(cat_rat) < 5):
        print("Rate {x} more!\n".format(x=5-len(cat_rat)))
    
but_layout = w.Layout(width='100%', height='100px')
but_items = [w.Button(description=c, layout=but_layout) for c in categories]
on_clk = [item.on_click(on_button_clicked) for item in but_items]
r1 = w.VBox([i for i in but_items[:6]])
r2 = w.VBox([i for i in but_items[6:12]])
r3 = w.VBox([i for i in but_items[12:]])
buttons = w.HBox([r1,r2,r3])
print("Select and rate atleast 5 categories and rate them:")
display(buttons)

Select and rate atleast 5 categories and rate them:


food,_wine_&_nightlife


IntSlider(value=0, description='Rate', max=5)

Rate 4 more!

water_sports


IntSlider(value=0, description='Rate', max=5)

...

IntSlider(value=0, description='Rate', max=5)

cultural_&_theme_tours


IntSlider(value=0, description='Rate', max=5)

private_&_custom_tours


IntSlider(value=0, description='Rate', max=5)

In [7]:
user_name = re.sub(' ','_',name.value.lower())
province = re.sub(' ','_',place.value.lower())
(low,high) = tuple([float(i) for i in budget.value])
begin_date = start.value
end_date = end.value
# cat_rating = dict()
# for key, value in cat_rat.items():
#     cat_rating[key] = float(value.value)
# cat_rating

In [12]:
def f(row):
    avg_cat_rat = dict()
    for i in range(len(row['category'])):
        if row['category'][i] not in avg_cat_rat:
            avg_cat_rat[row['category'][i]] = [row['rating'][i]]
        else:
            avg_cat_rat[row['category'][i]].append(row['rating'][i])
    for key,value in avg_cat_rat.items():
        avg_cat_rat[key] = sum(value)/len(value)
    return avg_cat_rat

def sim_score(row):
    score = 0.0
    match = 0
    col1 = row['cat_rat']
    col2 = row['user_data']
    for key, value in col2.items():
        if key in col1:
            match+=1
            score += (value-col1[key])**2
    if match != 0:
        return ((math.sqrt(score)/match) + (len(col2) - match))
    else:
        return 100

In [15]:
def get_recc(cat_rating):
    util = Util()
    epochs = 50
    rows = 40000
    alpha = 0.01
    H = 128
    batch_size = 16
    dir= 'etl/'
    ratings, attractions = util.read_data(dir)
    ratings = util.clean_subset(ratings, rows)
    rbm_att, train = util.preprocess(ratings)
    num_vis =  len(ratings)
    rbm = RBM(alpha, H, num_vis)
    
    joined = ratings.set_index('attraction_id').join(attractions[["attraction_id","category"]].set_index("attraction_id")).reset_index('attraction_id')
    grouped = joined.groupby('user_id')
    category_df = grouped['category'].apply(list).reset_index()
    rating_df = grouped['rating'].apply(list).reset_index()
    cat_rat_df = category_df.set_index('user_id').join(rating_df.set_index('user_id'))
    cat_rat_df['cat_rat'] = cat_rat_df.apply(f,axis=1)
    cat_rat_df = cat_rat_df.reset_index()[['user_id','cat_rat']]
    
    cat_rat_df['user_data'] = [cat_rating for i in range(len(cat_rat_df))]
    cat_rat_df['sim_score'] = cat_rat_df.apply(sim_score, axis=1)
    user = cat_rat_df.sort_values(['sim_score']).values[0][0]
    print("Similar User: {u}".format(u=user))
    filename = "e"+str(epochs)+"_r"+str(rows)+"_lr"+str(alpha)+"_hu"+str(H)+"_bs"+str(batch_size)
    reco, weights, vb, hb = rbm.load_predict(filename,train,user)
    unseen, seen = rbm.calculate_scores(ratings, attractions, reco, user)
    rbm.export(unseen, seen, 'recommendations/'+filename, str(user))
    return filename, user, rbm_att

In [16]:
filename, user, rbm_att = get_recc(cat_rating)
recc_df = pd.read_csv('recommendations/'+filename+'/user{u}_unseen.csv'.format(u=user), index_col=0)

Reading the data
Extracting num_rows from ratings
Preprocessing the dataset
Similar User: 15
Model restored


,attraction_id,Recommendation Score
0,0,0.018641
1,1,0.003147
2,2,0.000258
3,3,0.000034
4,4,0.000369


In [8]:
filename = "e"+str(50)+"_r"+str(40000)+"_lr"+str(0.01)+"_hu"+str(128)+"_bs"+str(16)
recc_df = pd.read_csv('recommendations/'+filename+'/user{u}_unseen.csv'.format(u=15), index_col=0)
recc_df.columns = ['attraction_id', 'att_name', 'att_cat', 'att_price', 'score']
# display(cat_rating)
display(recc_df.head(20))

,attraction_id,att_name,att_cat,att_price,score
6,6,whistler_small-group_day_trip_from_vancouver,featured_tours_and_tickets,145.00,5.000000
576,1303,vancouver_city_sightseeing_tour,tours_&_sightseeing,80.00,4.382824
1463,3470,whistler_sasquatch_zipline,recommended_experiences,135.45,3.118515
5,5,niagara_falls_in_one_day:_deluxe_sightseeing_t...,featured_tours_and_tickets,204.42,2.487308
1407,3404,stanley_park_horse-drawn_tours,recommended_experiences,42.00,2.251432
1434,3431,flavors_of_old_montreal_walking_tour,recommended_experiences,68.00,2.135264
583,1310,"best_of_niagara_falls_tour_from_niagara_falls,...",tours_&_sightseeing,158.00,2.110758
913,2025,little_italy_walking_food_tour_in_vancouver,walking_&_biking_tours,73.00,2.034340
0,0,vancouver_city_sightseeing_tour,featured_tours_and_tickets,80.00,2.000344
1061,2258,niagara_falls_helicopter_tour,family_friendly,145.00,1.843741


In [9]:
recommendation = att_df[['attraction_id','name','category','city','latitude','longitude','price','province','rating']].set_index('attraction_id').join(recc_df[['attraction_id','score']].set_index('attraction_id'), how="inner").reset_index().sort_values("score",ascending=False)
filtered = recommendation[(recommendation.province == province) & (recommendation.price >= low) & (recommendation.price >= low)]

In [28]:
url = pd.read_json('outputs/attractions_cat.json',orient='records')
url['id'] = url.index
with_url = filtered.set_index('attraction_id').join(url[['id','attraction']].set_index('id'), how="inner")
with_url.head()

,name,category,city,latitude,longitude,price,province,rating,score,attraction
5,niagara_falls_in_one_day:_deluxe_sightseeing_t...,featured_tours_and_tickets,niagara_falls,43.102436,-78.961639,204.42,ontario,5.0,2.487308,https://tripadvisor.ca/AttractionProductDetail...
1310,"best_of_niagara_falls_tour_from_niagara_falls,...",tours_&_sightseeing,niagara_falls,43.085712,-79.082428,158.00,ontario,5.0,2.110758,https://tripadvisor.ca/AttractionProductDetail...
2258,niagara_falls_helicopter_tour,family_friendly,niagara_falls,43.118603,-79.074387,145.00,ontario,4.5,1.843741,https://tripadvisor.ca/AttractionProductDetail...
3285,niagara_falls_day_tour_from_toronto,recommended_experiences,toronto,43.703075,-79.406387,94.92,ontario,5.0,1.734268,https://tripadvisor.ca/AttractionProductDetail...
2247,niagara_falls_day_and_evening_tour_with_boat_c...,family_friendly,toronto,43.683743,-79.592911,169.00,ontario,4.5,1.617617,https://tripadvisor.ca/AttractionProductDetail...


In [38]:
def get_image(name):
    name = name.split(",")[0]
    response = google_images_download.googleimagesdownload()
    args_list = ["keywords", "keywords_from_file", "prefix_keywords", "suffix_keywords",
             "limit", "format", "color", "color_type", "usage_rights", "size",
             "exact_size", "aspect_ratio", "type", "time", "time_range", "delay", "url", "single_image",
             "output_directory", "image_directory", "no_directory", "proxy", "similar_images", "specific_site",
             "print_urls", "print_size", "print_paths", "metadata", "extract_metadata", "socket_timeout",
             "thumbnail", "language", "prefix", "chromedriver", "related_images", "safe_search", "no_numbering",
             "offset", "no_download"]
    args = {}
    for i in args_list:
        args[i]= None
    args["keywords"] = name
    args['limit'] = 1
    params = response.build_url_parameters(args)
    url = 'https://www.google.com/search?q=' + quote(name) + '&espv=2&biw=1366&bih=667&site=webhp&source=lnms&tbm=isch' + params + '&sa=X&ei=XosDVaCXD8TasATItgE&ved=0CAcQ_AUoAg'
    try:
        response.download(args)
        for filename in glob.glob("downloads/{name}/*.jpg".format(name=name)):
            return filename
    except:
        for filename in glob.glob("downloads/*.png"):
            return filename


def top_recc(with_url):
    i=0
    while(1):
        first_recc = with_url.iloc[[i]]
        if(first_recc['name'].values.T[0] not in final['name']):
            final['name'].append(first_recc['name'].values.T[0])
            final['location'].append(first_recc[['latitude','longitude']].values.tolist()[0])
            final['price'].append(first_recc['price'].values.T[0])
            final['rating'].append(first_recc['rating'].values.T[0])
            final['image'].append(get_image(first_recc['name'].values.T[0]))
            final['category'].append(first_recc['category'].values.T[0])
            return
        else:
            i+=1

def find_closest(loc,tod):
    distance = list()
    for i in with_url[['latitude','longitude']].values.tolist():
        distance.append(math.sqrt((loc[0]-i[0])**2 + (loc[1]-i[1])**2))
    with_dist = with_url
    with_dist["distance"] = distance
    sorted_d = with_dist.sort_values('distance')
    if tod == "Evening":
        mask = sorted_d.name.apply(lambda x: any(j in x for j in evening))
        sorted_d  = sorted_d[mask]
    top_recc(sorted_d)
    
final = dict()
final['timeofday'] = []
final['image'] = []
final['name'] = []
final['location'] = []
final['price'] = []
final['rating'] = []
final['category'] = []

for i in range(1,(end_date - begin_date).days+1):
    for j in range(2):
        final['timeofday'].append('Morning')
    for j in range(2):
        final['timeofday'].append('Evening')

syns1 = wordnet.synsets("evening")
syns2 = wordnet.synsets("night")

evening = [word.lemmas()[0].name() for word in syns1] + [word.lemmas()[0].name() for word in syns2]
print(evening)
for i in range(len(final['timeofday'])): 
    if i%4 == 0: 
        top_recc(with_url)
    else:
        print(final['timeofday'][i])
        find_closest(final['location'][-1],final['timeofday'][i])

['evening', 'evening', 'evening', 'flush', 'even', 'even', 'night', 'night', 'night', 'night', 'night', 'night', 'night', 'Nox']

Item no.: 1 --> Item name = niagara_falls_in_one_day:_deluxe_sightseeing_tour_of_american_and_canadian_sides
Evaluating...
Morning

Item no.: 1 --> Item name = niagara_falls_canadian_side_tour_and_maid_of_the_mist_boat_ride
Evaluating...
Starting Download...
Completed Image ====> 1. niagara-falls-canadian.jpg

Errors: 0

Evening

Item no.: 1 --> Item name = 6-hour_niagara_falls_canadian_side_evening_illuminations_tour
Evaluating...
Starting Download...
Completed Image ====> 1. 6-hour-niagara-falls.jpg

Errors: 0

Evening

Item no.: 1 --> Item name = day_and_night_tour_of_niagara_falls
Evaluating...
Starting Download...
Completed Image ====> 1. day-and-night-tour-of-niagara-falls-in-niagara-falls-193929.jpg

Errors: 0


Item no.: 1 --> Item name = best_of_niagara_falls_tour_from_niagara_falls
Evaluating...
Starting Download...
Completed Image ====> 1. best-of

In [53]:
days = (end_date - begin_date).days
time = ['MORNING', 'EVENING']
fields = ['NAME', 'CATEGORY', 'LOCATION', 'PRICE', 'RATING']
recommendations = ['Recommendation 1:', 'Recommendation 2:']

box_layout = Layout(justify_content='space-between',
                    display='flex',
                    flex_flow='row', 
                    align_items='stretch',
                   )
column_layout = Layout(justify_content='space-between',
                    width='75%',
                    display='flex',
                    flex_flow='column', 
                   )
tab = []
for i in range(days):
    images = final['image'][i*4:(i+1)*4]
    images = [open(i, "rb").read() for i in images]
    name = [re.sub('_',' ',i).capitalize() for i in final['name'][i*4:(i+1)*4]]
    category = [re.sub('_',' ',i).capitalize() for i in final['category'][i*4:(i+1)*4]]
    location = ["("+str(i[0])+","+str(i[1])+")" for i in final['location'][i*4:(i+1)*4]]
    price = [str(i) for i in final['price'][i*4:(i+1)*4]]
    rating = [str(i) for i in final['rating'][i*4:(i+1)*4]]
    tab.append(VBox(children=
                    [HBox(children=
                          [VBox(children=
                                [widgets.HTML(value = f"<b><font color='orange'>{time[0]}</b>"),
                                 widgets.HTML(value = f"<b><font color='purple'>{recommendations[0]}</b>"),
                                 widgets.Image(value=images[0], format='jpg', width=300, height=400), 
                                 widgets.HTML(description=fields[0], value=f"<b><font color='black'>{name[0]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[1], value=f"<b><font color='black'>{category[0]}</b>", disabled=True),
                                 widgets.HTML(description=fields[2], value=f"<b><font color='black'>{location[0]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[3], value=f"<b><font color='black'>{price[0]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[4], value=f"<b><font color='black'>{rating[0]}</b>", disabled=True)
                                ], layout=column_layout), 
                            VBox(children=
                                [widgets.HTML(value = f"<b><font color='orange'>{time[1]}</b>"), 
                                 widgets.HTML(value = f"<b><font color='purple'>{recommendations[0]}</b>"),
                                 widgets.Image(value=images[2], format='png', width=300, height=400), 
                                 widgets.HTML(description=fields[0], value=f"<b><font color='black'>{name[2]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[1], value=f"<b><font color='black'>{category[2]}</b>", disabled=True),
                                 widgets.HTML(description=fields[2], value=f"<b><font color='black'>{location[2]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[3], value=f"<b><font color='black'>{price[2]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[4], value=f"<b><font color='black'>{rating[2]}</b>", disabled=True)
                                ], layout=column_layout)
                          ], layout=box_layout),
                     
                     HBox(children=
                          [VBox(children=
                                [widgets.HTML(value = f"<b><font color='purple'>{recommendations[1]}</b>"),
                                 widgets.Image(value=images[1], format='png', width=300, height=400), 
                                 widgets.HTML(description=fields[0], value=f"<b><font color='black'>{name[1]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[1], value=f"<b><font color='black'>{category[1]}</b>", disabled=True),
                                 widgets.HTML(description=fields[2], value=f"<b><font color='black'>{location[1]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[3], value=f"<b><font color='black'>{price[1]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[4], value=f"<b><font color='black'>{rating[1]}</b>", disabled=True)
                                ], layout=column_layout), 
                            VBox(children=
                                [widgets.HTML(value = f"<b><font color='purple'>{recommendations[1]}</b>"),
                                 widgets.Image(value=images[3], format='png', width=300, height=400), 
                                 widgets.HTML(description=fields[0], value=f"<b><font color='black'>{name[3]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[1], value=f"<b><font color='black'>{category[3]}</b>", disabled=True),
                                 widgets.HTML(description=fields[2], value=f"<b><font color='black'>{location[3]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[3], value=f"<b><font color='black'>{price[3]}</b>", disabled=True), 
                                 widgets.HTML(description=fields[4], value=f"<b><font color='black'>{rating[3]}</b>", disabled=True)
                                ], layout=column_layout),
                          ], layout=box_layout)
                    
                    ]))
    
tab_recc = widgets.Tab(children=tab)
for i in range(len(tab_recc.children)):
    tab_recc.set_title(i, str('Day '+ str(i+1)))
tab_recc